In [139]:
using Printf
using Random
Random.seed!(1234)
m = 3
n = 4
A = rand(1:3, (m,n))
b = rand(1:3,m)
print(A)
println()
print(b)

[1 3 3 3; 2 2 3 2; 1 2 2 3]
[1, 1, 2]

### Idea
---
$$
\begin{gather}
    A' = \begin{bmatrix} A &  b \end{bmatrix} \\
    U' = CA' = \begin{bmatrix} CA &  Cb \end{bmatrix} \\
    U' = \begin{bmatrix} U &  Cb \end{bmatrix} \\
\end{gather}
$$
If $Cb$ is a pivot column, then the system is unsolvable, else it is consistent and has one or more solutions.

In [140]:
function rankconsistency(A::Matrix{Int64}, b::Vector{Int64})
    consistent::Bool = false
    U = rowreduce(A)
    return Bool,U  
end 


rankconsistency (generic function with 1 method)

In [141]:
function AddGf(i, j)
Add_matrix = [  0 1 2 3  ;  1  0  3  2 ; 2 3 0 1 ; 3 2 1 0 ]
    return Add_matrix[i+1, j+1]
end

AddGf (generic function with 1 method)

In [142]:
function MulGf(i, j)
    Mul_matrix = [ 0 0 0 0 ; 0  1  2  3 ; 0  2  3  1 ; 0 3 1 2 ]  
    return Mul_matrix[i+1, j+1]
end

MulGf (generic function with 1 method)

In [143]:
Inv_vec = [1, 3, 2]
function InvGf(i)
    return Inv_vec[i]
end

InvGf (generic function with 1 method)

In [144]:
function RowReduction(A::Matrix{Int64})

end

RowReduction (generic function with 1 method)

In [145]:
function PivotSubtract(A, pivot_row)
    for i in pivot_row + 1:size(A, 1)
        RowSubtract(A,pivot_row, i)
    end 
end

PivotSubtract (generic function with 1 method)

In [146]:
function RowSubtract(A, i, j)
    @printf("Subtracting Row %d form Row %d \n", i , j)
    factor = MulGf(InvGf(A[i , i]), A[j, i])
    @printf("factor of subtraction = %d \n",factor)
    for index in i:size(A, 2)
        A[j , index] = AddGf(A[j , index], MulGf( A[i, index] ,factor))
    end
end

RowSubtract (generic function with 1 method)

In [153]:
display(A)
PivotSubtract(A, 1)
PivotSubtract(A, 2)
display(A)


3×4 Matrix{Int64}:
 1  3  3  3
 0  3  2  3
 0  1  1  0

Subtracting Row 1 form Row 2 
factor of subtraction = 0 
Subtracting Row 1 form Row 3 
factor of subtraction = 0 
Subtracting Row 2 form Row 3 
factor of subtraction = 2 


3×4 Matrix{Int64}:
 1  3  3  3
 0  3  2  3
 0  0  2  1